In [1]:
import numpy as np
import pathlib
import os
import re
import math
import imageio
from skimage.metrics import structural_similarity as ssim
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import cv2  

In [2]:
# Set the folder path where images are located and the tile size
path = pathlib.Path(os.getcwd())
path_gt = path/'Hagen_testmix/gt_images'
path_noisy = path/'Hagen_testmix/noisy_images'
path_Base = path/'Hagen_testmix/Hagen_pred_images'
path_Transfer = path/'Hagen_testmix/Transfer_pred_images'
path_Transfer_all = path/'Hagen_testmix/Transfer_all_pred_images'
samples = ['actin-20x-noise1','actin-60x-noise1','actin-60x-noise2','mito-20x-noise1','mito-60x-noise1','mito-60x-noise2',
           'actin-confocal','mito-confocal','nucleus','membrane']

In [3]:
def natural_sort(l): 
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(l, key=alphanum_key)

In [42]:
# scale and shift as described by Weigert et al
def scale_and_shift(image1,image2):
    
    y = np.array(image2/(2**16-1))        # gt, already percentile normalized in image_to_tiles
    y_hat = np.array(image1/(2**16-1))
    
    def mse_fn(params):
        a, b = params
        y_hat_scaled = a*y_hat+b
        mse = np.mean((y-y_hat_scaled)**2)
        return mse

    initial_guess = [1.0, 0.0]
    boundaries = [(-20,20),(-1,1)]

    result = minimize(mse_fn,initial_guess,bounds=boundaries)

    if result.success:
        optimized_a,optimized_b = result.x
        y_hat_scaled = optimized_a*y_hat+optimized_b
        return optimized_a,optimized_b,y_hat_scaled
    else:
        raise Exception("optimization failed:" + result.message)

In [46]:
# applying scale and shift as in the paper by Hagen et al.
def calculate_ssim_and_psnr(image1_path, image2_path, scale=True):
    # Read the two images
    image1 = imageio.v2.imread(image1_path)
    image2 = imageio.v2.imread(image2_path)

    # Convert images to grayscale (if they are not already)
    if len(image1.shape) == 3:
        image1 = image1[:,:,0]
    if len(image2.shape) == 3:
        image2 = image2[:,:,0]

    if scale:    # scale and shift according to Weigert et al and Hagen et al
        a,b,image1 = scale_and_shift(image1,image2)
        image2 = image2/(2**16-1)
    else:
        image1 = image1/(2**16-1)
        image2 = image2/(2**16-1)
    
    # Calculate SSIM
    ssim_value = ssim(image1, image2, data_range=1)   
    
    # Calculate PSNR
    def calculate_psnr(img1, img2):
        # img1 and img2 have range [0, 1] after normalization
        img1 = img1.astype(np.float64)
        img2 = img2.astype(np.float64)
        mse = np.mean((img1 - img2)**2)
        if mse == 0:
            return float('inf')
        return 10 * math.log10(1 / mse)                        # like in Hagen et al           
    
    psnr_value = calculate_psnr(image1, image2)

    return ssim_value, psnr_value

In [47]:
# get metrics for original noise images - derived as published
all_ssim = []
all_psnr = []
for i,sample in enumerate(samples):
    gt = natural_sort(os.listdir(path_gt/sample))    
    for j,fn in enumerate(gt):
       temp1, temp2 = calculate_ssim_and_psnr(path_noisy/sample/fn,path_gt/sample/fn,scale=True)
       all_ssim.append(temp1)
       all_psnr.append(temp2)
       if j==len(gt)-1:
           print(f'{sample} - mean psnr: {np.mean(all_psnr)} and ssim: {np.mean(all_ssim)}')
           all_ssim = []
           all_psnr = []

actin-20x-noise1 - mean psnr: 24.182226401084524 and ssim: 0.34010388845896317
actin-60x-noise1 - mean psnr: 28.018827627070557 and ssim: 0.5304814135816318
actin-60x-noise2 - mean psnr: 18.419397063476794 and ssim: 0.17301112244882638
mito-20x-noise1 - mean psnr: 24.56299345060747 and ssim: 0.3535035408820367
mito-60x-noise1 - mean psnr: 27.9901587663801 and ssim: 0.5111835995572737
mito-60x-noise2 - mean psnr: 20.070719179734148 and ssim: 0.27942751489825485
actin-confocal - mean psnr: 24.720339297032304 and ssim: 0.5360678350740622
mito-confocal - mean psnr: 22.153394978903798 and ssim: 0.36869273970545297
nucleus - mean psnr: 24.717197906773343 and ssim: 0.38500256763880925
membrane - mean psnr: 29.468907158408722 and ssim: 0.6055796458614249


In [48]:
# get metrics for Base model predictions
all_ssim = []
all_psnr = []
for i,sample in enumerate(samples):
    gt = natural_sort(os.listdir(path_gt/sample))    
    for j,fn in enumerate(gt):
       temp1, temp2 = calculate_ssim_and_psnr(path_Base/sample/fn,path_gt/sample/fn,scale=True)
       all_ssim.append(temp1)
       all_psnr.append(temp2)
       if j==len(gt)-1:
           print(f'{sample} - mean psnr: {np.mean(all_psnr)} and ssim: {np.mean(all_ssim)}')
           all_ssim = []
           all_psnr = []

actin-20x-noise1 - mean psnr: 30.96925860061407 and ssim: 0.852369390159625
actin-60x-noise1 - mean psnr: 37.3599160164852 and ssim: 0.9369299803679553
actin-60x-noise2 - mean psnr: 25.98550279371081 and ssim: 0.37660686616304656
mito-20x-noise1 - mean psnr: 31.001375266968118 and ssim: 0.8671067178815471
mito-60x-noise1 - mean psnr: 35.437710455404364 and ssim: 0.916340112839541
mito-60x-noise2 - mean psnr: 25.40079377926142 and ssim: 0.45936460402647894
actin-confocal - mean psnr: 26.774895445107184 and ssim: 0.6583979555378257
mito-confocal - mean psnr: 26.354641927265448 and ssim: 0.5936610323601016
nucleus - mean psnr: 32.886455733822544 and ssim: 0.8230383899772076
membrane - mean psnr: 26.390798735601788 and ssim: 0.700782708753576


In [49]:
# get metrics for Transfer model predictions
all_ssim = []
all_psnr = []
for i,sample in enumerate(samples):
    gt = natural_sort(os.listdir(path_gt/sample))    
    for j,fn in enumerate(gt):
       temp1, temp2 = calculate_ssim_and_psnr(path_Transfer/sample/fn,path_gt/sample/fn,scale=True)
       all_ssim.append(temp1)
       all_psnr.append(temp2)
       if j==len(gt)-1:
           print(f'{sample} - mean psnr: {np.mean(all_psnr)} and ssim: {np.mean(all_ssim)}')
           all_ssim = []
           all_psnr = []

actin-20x-noise1 - mean psnr: 31.1558460177804 and ssim: 0.8533185278848364
actin-60x-noise1 - mean psnr: 37.18237011820437 and ssim: 0.9342779377375436
actin-60x-noise2 - mean psnr: 25.528760712283642 and ssim: 0.41558914545521386
mito-20x-noise1 - mean psnr: 31.156383310311224 and ssim: 0.8666675001313242
mito-60x-noise1 - mean psnr: 36.28286911728979 and ssim: 0.9265478216947596
mito-60x-noise2 - mean psnr: 25.054041972385335 and ssim: 0.4296295498342161
actin-confocal - mean psnr: 26.777906284365226 and ssim: 0.6447100876682481
mito-confocal - mean psnr: 25.964845490280283 and ssim: 0.5549174238519694
nucleus - mean psnr: 32.889848241152166 and ssim: 0.818500065776732
membrane - mean psnr: 30.875898064903897 and ssim: 0.7764634722846614


In [50]:
# get metrics for Transfer model predictions
all_ssim = []
all_psnr = []
for i,sample in enumerate(samples):
    gt = natural_sort(os.listdir(path_gt/sample))    
    for j,fn in enumerate(gt):
       temp1, temp2 = calculate_ssim_and_psnr(path_Transfer_all/sample/fn,path_gt/sample/fn,scale=True)
       all_ssim.append(temp1)
       all_psnr.append(temp2)
       if j==len(gt)-1:
           print(f'{sample} - mean psnr: {np.mean(all_psnr)} and ssim: {np.mean(all_ssim)}')
           all_ssim = []
           all_psnr = []

actin-20x-noise1 - mean psnr: 32.03840591725487 and ssim: 0.8939877509776692
actin-60x-noise1 - mean psnr: 37.13776365613062 and ssim: 0.9394228356898008
actin-60x-noise2 - mean psnr: 28.517065691280823 and ssim: 0.8024370869225026
mito-20x-noise1 - mean psnr: 32.730079630080176 and ssim: 0.9189335023373463
mito-60x-noise1 - mean psnr: 37.83894219525592 and ssim: 0.95417317132584
mito-60x-noise2 - mean psnr: 27.80357488396706 and ssim: 0.8414408238185704
actin-confocal - mean psnr: 28.399907610259888 and ssim: 0.8177768531907466
mito-confocal - mean psnr: 26.20430408355928 and ssim: 0.6881366830584542
nucleus - mean psnr: 34.707298484096334 and ssim: 0.8704846185685899
membrane - mean psnr: 35.208937614166885 and ssim: 0.9235279675236173
